In [1]:
%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt

import time
import numpy as np

from SES_setup import *
from utils import *
from style import *

SES_setup loaded globally


In [2]:
#import weak_phonons as wp

T_ph = 300.
w_2 = 3500. #1.4*ev_to_inv_cm
bias = w_2*0.01 #0.01*ev_to_inv_cm #0.0000001*ev_to_inv_cm
V = w_2*0.01 #0.01*ev_to_inv_cm #0.00001*ev_to_inv_cm
alpha = 1.
alpha_EM = 5.309e-3 # inv_ps_to_inv_cm *10^-3
N =4
#wc = 50.
w_0 = 100.
Gamma = 2*w_0
T_EM = 2000.

#phonons = np.logspace(-1,np.log10(20),8)

PARAMS = PARAMS_setup(bias=bias, w_2=w_2, V = V, alpha=alpha,
                      T_EM=T_EM, T_ph =T_ph, alpha_EM=alpha_EM, shift=True,
                      num_cpus=4, N=N, Gamma=Gamma, w_0=w_0,
                      silent=True, exc_diff=0)
#print PARAMS['H_sub'].eigenstates()
H, L, L_RWA, PARAMS = get_H_and_L_RWA(PARAMS,silent=False, threshold=0.)


evals, evecs = exciton_states(PARAMS, shift=False)
dark, bright = evecs[0]*evecs[0].dag(), evecs[1]*evecs[1].dag()
eig_x = (evecs[0]*evecs[1].dag() + evecs[1]*evecs[0].dag())*0.5
ops = make_expectation_operators(PARAMS)
site_x = (site_coherence+site_coherence.dag())
print( PARAMS['H_sub'])

H, L, L_RWA, PARAMS = get_H_and_L_RWA(PARAMS, silent=True, threshold=1e-6)

****************************************************************
The operators took 1.789353847503662 and have dimension 39.
Building the RC Liouvillian took 1.836 seconds.
It is 1521by1521. The full basis would be 2304by2304
It took  0.6616377830505371  seconds to build the Non-secular RWA Liouvillian
Full optical Liouvillian took 0.5699288845062256 seconds.
Chopping reduced the sparsity from 10.104% to 10.104%
Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[   0.    0.    0.]
 [   0. 3535.   35.]
 [   0.   35. 3500.]]


In [61]:
qt.enr_destroy([3,3], 3)

[Quantum object: dims = [[3, 3], [3, 3]], shape = (8, 8), type = oper, isherm = False
 Qobj data =
 [[0.         0.         0.         1.         0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         1.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         1.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   1.41421356 0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         1.41421356]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]],
 Quantum object: dims = [[3, 3], [3, 3]], shape = (8, 8), type = oper, isherm = False
 Qobj data =
 [[0.         1.         0.         0.         0.         0.
   0.         0.        ]
  [0.         0. 

In [ ]:
# for this comparison, we need to look at two state spaces with the same size, but show that the one that
# employs ENR uses more favourable states (higher entropy, more likelihood)

In [67]:
def ENR_basis_labels(N_1, N_2, exc):
    state_space = set()
    for n1 in range(N_1):
        for n2 in range(N_2):
            state_name  = '|{},{}|'.format(n1, n2)
            #print (n1+n2)
            
            if (n1+n2)<=exc:
                state_space.add(state_name)
    assert qt.enr_destroy([N_1,N_2], exc)[0].shape[0] ==  len(state_space) 
    return state_space



In [72]:
print(ENR_basis_labels(5, 5, 5).difference(ENR_basis_labels(5, 5, 4)))
print(ENR_basis_labels(5, 5, 4).difference(ENR_basis_labels(4, 4, 4)))

{'|4,1|', '|3,2|', '|1,4|', '|2,3|'}
{'|4,0|', '|0,4|'}


In [ ]:


dims = []
N_exc = []
state_spaces = []
for n in range(3,7):
    for exc in range(n, 2*n+1):
        stsp = ENR_basis_labels(n, n, exc)
        
        dims.append(len(stsp))
        state_spaces.append(stsp)
        N_exc.append((n,exc))

print(state_spaces[0], '\n',state_spaces[1], '\n',state_spaces[2], '\n',state_spaces[3])
idx1 = 1
idx2 = 2
print(N_exc)
print(dims)
base = state_spaces[idx1]
base_ENR = state_spaces[idx2]
print( base, '\n',base_ENR)
print(N_exc[idx1], N_exc[idx2])
print(dims)
print( len(base), len(base_ENR))
print( base.difference(base_ENR))
print(base_ENR.difference(base))
#print( base.difference(ENR_basis_labels(4, 4, 5)), ENR_basis_labels(4, 4, 5))